# First steps with OMEGA - Open box

An open box in chemical evolution refers to a gas reservoir which exchange matter with its surrounding.  In addition of the star formation and enrichment processes, the gas reservoir is altered by galactic inflows (gas introduced in the system) and galactic outflows (enriched gas expelled from the system).  This notebook presents in a general wah impact of galactic inflows and outflows.  Please take a look at the OMEGA_userguide notebook for more details on the different inflow and outflow implementations.

In [ ]:
# Python packages
%matplotlib nbagg
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.reset_orig()

# Shared class between SYGMA and OMEGA
import chem_evol
reload(chem_evol)

# One-zone galactic chemical evolution code
import omega
reload(omega)

# There may be warning messages here .. but it's all ok!

## 1. Galactic inflows

In a closed-box model, all the gas needed to sustain the star formation history (SFH) needs to be in the gas reservoir at the beginning of the simulation.  This sets a limit on the minimum mass of the gas reservoir.  However, with an open-box model and galactic inflows, it is possible reduce the initial mass of gas below that limit, as the **inflowing gas will fuel the star formation process at later times**.

### 1.1. Galactic inflows and appearance of SNe Ia

Below is an example of how galactic inflows can help to control the appearance of SNe Ia on the [Fe/H] axis (the so-called SN Ia knee).  To do so, let's run simulations with different gas reservoirs.

In [ ]:
# Run OMEGA simulations

# Constant star formation rate with different initial gas reservoir (closed box)
o_res_1e11_close = omega.omega(cte_sfr=1.0, mgal=1e11)
o_res_1e10_close = omega.omega(cte_sfr=1.0, mgal=1e10)
o_res_1e09_close = omega.omega(cte_sfr=1.0, mgal=1e9)  # This is not enough gas to sustain the SFH.

# To isolate where the SN Ia knee appears, each simulation is ran a second
# time without the contribution of SNe Ia using the sn1a_on parameter.
o_res_1e11_close_noIa = omega.omega(cte_sfr=1.0, mgal=1e11, sn1a_on=False)
o_res_1e10_close_noIa = omega.omega(cte_sfr=1.0, mgal=1e10, sn1a_on=False)
o_res_1e09_close_noIa = omega.omega(cte_sfr=1.0, mgal=1e9,  sn1a_on=False)  # Still not enough gas.

In [ ]:
# Plot the predicted chemical evolution
%matplotlib nbagg
yaxis = '[C/Fe]'
xaxis = '[Fe/H]'

# Initial gas mass of 1e11 Msun
o_res_1e11_close.plot_spectro(     fig=1, xaxis=xaxis, yaxis=yaxis, color='r', label='mgal = 1e11')
o_res_1e11_close_noIa.plot_spectro(fig=1, xaxis=xaxis, yaxis=yaxis, color='r', label='mgal = 1e11, no Ia', shape='--')
plt.annotate('Closed box', color='r', xy=(0.65, 0.8), xycoords='axes fraction', fontsize=13)
plt.xlim(-9,1.5)

# Initial gas mass of 1e10 Msun
o_res_1e10_close.plot_spectro(     fig=2, xaxis=xaxis, yaxis=yaxis, color='k', label='mgal = 1e10')
o_res_1e10_close_noIa.plot_spectro(fig=2, xaxis=xaxis, yaxis=yaxis, color='k', label='mgal = 1e10, no Ia', shape='--')
plt.annotate('Closed box', color='k', xy=(0.65, 0.8), xycoords='axes fraction', fontsize=13)
plt.xlim(-9,1.5)

# Initial gas mass of 1e09 Msun
o_res_1e09_close.plot_spectro(     fig=3, xaxis=xaxis, yaxis=yaxis, color='b', label='mgal = 1e9')
o_res_1e09_close_noIa.plot_spectro(fig=3, xaxis=xaxis, yaxis=yaxis, color='b', label='mgal = 1e9, no Ia', shape='--')
plt.annotate('Closed box', color='b', xy=(0.65, 0.8), xycoords='axes fraction', fontsize=13)
plt.xlim(-9,1.5)

**The initial mass of the gas reservoir has a significant impact on where SNe Ia first appear on the [Fe/H] axis**.  In the Milky Way, for example, the knee must occur at [Fe/H] = -1 to be consistent with observations.  As seen above, this can be achieved with a low-mass initial gas reservoir (blue lines).  But with a closed-box model, the star formation eventually stops because of the lack of gas.  When this happens, the star formation will feed from the stellar ejecta only and will generate unreliable predictions at high [Fe/H].  A solution is to include galactic inflows.

In [ ]:
# Run OMEGA simulations
# The in_out_control parameter allows to use an open box model.
# Please look at the OMEGA_userguide notebook for more options and different open-box implementations.

# Low-mass gas reservoir with galactic inflows (no galactic outflow for now)
o_res_1e09_open = omega.omega(cte_sfr=1.0, mgal=1e9, in_out_control=True, outflow_rate=0.0, inflow_rate=1.0)

# Without the contribution of SNe Ia
o_res_1e09_open_noIa = omega.omega(cte_sfr=1.0, mgal=1e9,\
    sn1a_on=False, in_out_control=True, outflow_rate=0.0, inflow_rate=1.0)

In [ ]:
# Plot the predicted chemical evolution
%matplotlib nbagg
o_res_1e09_open.plot_spectro(     xaxis=xaxis, yaxis=yaxis, color='b', label='mgal = 1e9')
o_res_1e09_open_noIa.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='b', label='mgal = 1e9, no Ia', shape='--')
plt.xlim(-9,1.5)

# Annotation and [Fe/H] range
plt.annotate('Open box', color='b', xy=(0.65, 0.8), xycoords='axes fraction', fontsize=13)

As show in the plot above using an open-box model, the SN Ia knee still occurs at [Fe/H] = -1, but the system does not run out of gas anymore.  The predictions at high [Fe/H] are now reliable.

### 1.2. Dilution process 

Galactic inflows typically have a lower metallicity than the gas inside galaxies.  Therefore, they tend to dilute the metal concentration of the galactic gas and to slow down the gas enrichment process.  In the following examples, galactic inflows have primordial composition (only H, He, and Li).

In [ ]:
# Run OMEGA simulations.
# Constant star formation rate with different inflow rate
o_in_rate_0_0 = omega.omega(cte_sfr=1.0, mgal=2e10, in_out_control=True, outflow_rate=0.0, inflow_rate=0.0)
o_in_rate_1_0 = omega.omega(cte_sfr=1.0, mgal=2e10, in_out_control=True, outflow_rate=0.0, inflow_rate=1.0)
o_in_rate_3_0 = omega.omega(cte_sfr=1.0, mgal=2e10, in_out_control=True, outflow_rate=0.0, inflow_rate=3.0)
o_in_rate_9_0 = omega.omega(cte_sfr=1.0, mgal=2e10, in_out_control=True, outflow_rate=0.0, inflow_rate=9.0)

In [ ]:
# Plot the mass of H, Ca, and Fe in the galactic gas as a function of time.
%matplotlib nbagg
o_in_rate_9_0.plot_mass(specie='H',  color='b', label='inflow_rate = 9.0 Msun/yr')
o_in_rate_3_0.plot_mass(specie='H',  color='r', label='inflow_rate = 3.0')
o_in_rate_1_0.plot_mass(specie='H',  color='g', label='inflow_rate = 1.0')
o_in_rate_0_0.plot_mass(specie='H',  color='m', label='inflow_rate = 0.0')
o_in_rate_9_0.plot_mass(specie='Ca', color='b', show_legend=False)
o_in_rate_3_0.plot_mass(specie='Ca', color='r', show_legend=False)
o_in_rate_1_0.plot_mass(specie='Ca', color='g', show_legend=False)
o_in_rate_0_0.plot_mass(specie='Ca', color='m', show_legend=False)
o_in_rate_9_0.plot_mass(specie='Fe', color='b', show_legend=False)
o_in_rate_3_0.plot_mass(specie='Fe', color='r', show_legend=False)
o_in_rate_1_0.plot_mass(specie='Fe', color='g', show_legend=False)
o_in_rate_0_0.plot_mass(specie='Fe', color='m', show_legend=False)
plt.xscale('linear')
plt.ylim(1e3,3e11)

# Add element annotations.
plt.annotate('H',  xy=(0.2e10, 3.0e9))
plt.annotate('Ca', xy=(1.0e10, 1.5e5))
plt.annotate('Fe', xy=(1.0e10, 3.0e7))

Because the galactic inflows are primordial, Ca and Fe are not affected by them.  This means the evolution of [Ca/Fe] is not altered, as opposed to the evolution of [Fe/H], as shown below.

In [ ]:
# Plot the evolution of [Fe/H] as a function of time.
%matplotlib nbagg
o_in_rate_0_0.plot_spectro(color='m', label='inflow_rate = 0.0 Msun/yr')
o_in_rate_1_0.plot_spectro(color='g', label='inflow_rate = 1.0')
o_in_rate_3_0.plot_spectro(color='r', label='inflow_rate = 3.0')
o_in_rate_9_0.plot_spectro(color='b', label='inflow_rate = 9.0')
plt.ylim(-3, 0.5)

Galactic inflows dilute the metal concentration of the galactic gas, especially at late times, as the evolution of [Fe/H] at early times mainly depends on the initial mass of gas.  **In the [X/Fe] vs [Fe/H] space, introducing galactic inflows shrinks the predictions and shifts the final [Fe/H] to lower values**.

In [ ]:
# Plot the predicted chemical evolution.
%matplotlib nbagg
yaxis = '[Ca/Fe]'
xaxis = '[Fe/H]'
o_in_rate_0_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='m', label='inflow_rate = 0.0 Msun/yr')
o_in_rate_1_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='g', label='inflow_rate = 1.0')
o_in_rate_3_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='r', label='inflow_rate = 3.0')
o_in_rate_9_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='b', label='inflow_rate = 9.0')
plt.xlim(-4.0, 0.5)
plt.ylim(-0.2, 0.3)

### 1.3. Exercise with SYGMA

As an exercise, you can use SYGMA to investigate why there is a drop in [Ca/Fe] at [Fe/H] = -0.15 in the previous plot (pink line).

In [ ]:
# Import the simple stellar population (SSP) code SYGMA
import sygma
reload(sygma)
s_0_02  = sygma.sygma(iniZ=0.02)
s_0_01  = sygma.sygma(iniZ=0.01)
s_0_006 = sygma.sygma(iniZ=0.006)
s_0_001 = sygma.sygma(iniZ=0.001)

In [ ]:
# Plot the evolution of [Ca/Fe] of the cumulated SSP ejecta (no other gas mixture)
yaxis='[Ca/Fe]'
%matplotlib nbagg
s_0_02.plot_spectro( yaxis=yaxis, color='b', label='Z = 0.02')
s_0_01.plot_spectro( yaxis=yaxis, color='r', label='Z = 0.01')
s_0_006.plot_spectro(yaxis=yaxis, color='g', label='Z = 0.006')
s_0_001.plot_spectro(yaxis=yaxis, color='m', label='Z = 0.001')
plt.annotate('SSP - SYGMA',  xy=(0.6, 0.8), xycoords='axes fraction')
plt.ylim(-0.8,0.8)

As shown by SYGMA, the overall [Ca/Fe] ratios ejected by SSPs are similar for Z = 0.001 and 0.006 (pink and green lines), while it is about 0.1 dex lower for Z = 0.01 (red line).  The drop seen in the GCE predictions is thus caused by the transition between Z = 0.006 yields and Z = 0.01 yields.  Let's see which stars are responsible for this feature.

In [ ]:
# Plot the contribution of individual stars on the production of Ca.
%matplotlib nbagg
specie = 'Ca'
s_0_01.plot_mass_range_contributions( specie=specie, color='r', label='Ca, Z = 0.01')
s_0_006.plot_mass_range_contributions(specie=specie, color='g', label='Ca, Z = 0.006')
plt.annotate('SNe Ia not shown',  xy=(0.5, 0.8), xycoords='axes fraction')

In [ ]:
# Plot the contribution of individual stars on the production of Fe.
%matplotlib nbagg
specie = 'Fe'
s_0_01.plot_mass_range_contributions( specie=specie, color='r', label='Fe, Z = 0.01')
s_0_006.plot_mass_range_contributions(specie=specie, color='g', label='Fe, Z = 0.006')
plt.annotate('SNe Ia not shown',  xy=(0.5, 0.8), xycoords='axes fraction')

When we go from Z = 0.006 to Z = 0.01, the total Fe ejected by massive stars is relatively similar, while there is a significant drop of Ca ejecta in the 15 and 20 Msun stellar models.  Even if further exploration is needed to understand why this feature occurs, this demonstrates that **SYGMA can efficiently isolate the source of peculiar behaviors seen with OMEGA**.  This last plot does not show the contribution of SNe Ia.

## 2. Star formation with inflows and outflows

In the field of galaxy evolution, it is common to connect the SFH with the mass of the gas reservoir at each timestep.  This idea comes from the empirical relation between the star formation rate and the gas surface density of disc galaxies (e.g., Schmidt 1959; Kennicutt 1998).

### 2.1. Mass of gas with the SF_law parameter

With OMEGA, this connection is achieved using the *SF_law* parameter (see also the *DM_evolution* parameter in the OMEGA_userguide notebook).

In [ ]:
# Run OMEGA simulations without and with the SF_law parameter
o_without_SF = omega.omega(cte_sfr=1.0, mgal=1.0e10)
o_with_SF    = omega.omega(cte_sfr=1.0, SF_law=True)

In [ ]:
# Plot the mass of the gas reservoir
%matplotlib nbagg
o_without_SF.plot_totmasses(color='r', label='Without SF_law')
o_with_SF.plot_totmasses(   color='b', label='With SF_law')
plt.ylim(8e8, 2e10)

**When SF_law = True, the temporal profile of the gas reservoir follows the shape of the SFH**.  If the SFH is constant in time, the mass of the galactic gas ($M_{gas}$) will also be constant in time.  However, the gas reservoir profile can be scalled with the *sfe* parameter, which represents the star formation efficiency.

$$M_{gas}(t)=\frac{SFH(t)}{sfe}$$

A high star formation efficiency implies that a small gas reservoir is sufficient to form stars, while a low star formation efficiency will require a larger gas reservoir to form the same amount of stars.

In [ ]:
# Run OMEGA simulations with different star formation efficiencies
o_SF_sfe_0_01 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.01)
o_SF_sfe_0_04 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.04)
o_SF_sfe_0_16 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.16)

In [ ]:
# Plot the mass of the gas reservoir
%matplotlib nbagg
o_SF_sfe_0_01.plot_totmasses(color='m', label='sfe = 0.01')
o_SF_sfe_0_04.plot_totmasses(color='c', label='sfe = 0.04')
o_SF_sfe_0_16.plot_totmasses(color='g', label='sfe = 0.16')
plt.ylim(4e8, 3e10)

### 2.2. Galactic inflows and outflows

When using the *SF_law* parameter, galactic inflows and outflows operate in a symbiotic manner.  For example, if a lot of mass is ejected by galactic outflows, a lot of mass will also be introduced by galactic inflows in order to preserve the required mass of gas to sustain the SFH.  **The intensity of the inflow/outflow circulation plays a crucial role in the metal concentration of the galactic gas**, as shown below.  Within this framework, the outflow rate is defined by multiplying the SFH with the *mass_loading* parameter, a dimensionless proportionality constant.  The inflow rate is automatically calculated.

In [ ]:
# Run OMEGA simulations with different outflow rates (and thus different inflow rates)
o_SF_ml_0_0 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.1, mass_loading=0.0)
o_SF_ml_1_0 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.1, mass_loading=1.0)
o_SF_ml_3_0 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.1, mass_loading=3.0)
o_SF_ml_9_0 = omega.omega(cte_sfr=1.0, SF_law=True, sfe=0.1, mass_loading=9.0)

In [ ]:
# Plot the mass of H, Ca, and Fe in the galactic gas as a function of time.
%matplotlib nbagg
o_SF_ml_0_0.plot_mass(specie='H',  color='b', label='mass_loading = 0.0')
o_SF_ml_1_0.plot_mass(specie='H',  color='r', label='mass_loading = 1.0')
o_SF_ml_3_0.plot_mass(specie='H',  color='g', label='mass_loading = 3.0')
o_SF_ml_9_0.plot_mass(specie='H',  color='m', label='mass_loading = 9.0')
o_SF_ml_0_0.plot_mass(specie='Fe', color='b', show_legend=False)
o_SF_ml_1_0.plot_mass(specie='Fe', color='r', show_legend=False)
o_SF_ml_3_0.plot_mass(specie='Fe', color='g', show_legend=False)
o_SF_ml_9_0.plot_mass(specie='Fe', color='m', show_legend=False)
plt.xscale('linear')
plt.ylim(1e4,1e10)

# Add element annotations.
plt.annotate('H',  xy=(0.2e10, 3.0e8))
plt.annotate('Fe', xy=(1.0e10, 5.0e6))

Because SF_law is used with a constant SFH, the mass of the gas reservoir (mostly H) stays constant with time.  However, more powerful outflows (higher *mass_loading* parameter) implies that metals (e.g., Fe) will be lost from the system and will be replaced by primordial gas (at least in our case).  **In the [X/Fe] vs [Fe/H] space, as shown in Section 2.1, the intensity of the gas circulation shrinks the predictions and shifts the final [Fe/H] to lower values**.

In [ ]:
# Plot the predicted chemical evolution.
%matplotlib nbagg
yaxis = '[Mn/Fe]'
xaxis = '[Fe/H]'
o_SF_ml_0_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='b', label='mass_loading = 0.0')
o_SF_ml_1_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='r', label='mass_loading = 1.0')
o_SF_ml_3_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='g', label='mass_loading = 3.0')
o_SF_ml_9_0.plot_spectro(xaxis=xaxis, yaxis=yaxis, color='m', label='mass_loading = 9.0')
plt.xlim(-5, 0.5)

### 2.3. Exercise with SYGMA

As an exercise, you can use SYGMA to investigate why, in the last plot, the final [Mn/Fe] value decreases when the galactic outflow rate (defined by the *mass_loading* parameter) increases.

In [ ]:
# Import the simple stellar population (SSP) code SYGMA
import sygma
reload(sygma)
s_0_01   = sygma.sygma(iniZ=0.01)
s_0_006  = sygma.sygma(iniZ=0.006)
s_0_001  = sygma.sygma(iniZ=0.001)
s_0_0001 = sygma.sygma(iniZ=0.0001)

In [ ]:
# Plot the evolution of [Mn/Fe] of the cumulated SSP ejecta (no other gas mixture)
%matplotlib nbagg
s_0_01.plot_spectro(  yaxis=yaxis, color='r', label='Z = 0.01')
s_0_006.plot_spectro( yaxis=yaxis, color='g', label='Z = 0.006')
s_0_001.plot_spectro( yaxis=yaxis, color='b', label='Z = 0.001')
s_0_0001.plot_spectro(yaxis=yaxis, color='m', label='Z = 0.0001')
plt.annotate('SSP - SYGMA', xy=(0.6, 0.8), xycoords='axes fraction')

From the plot above, we see that the final [Mn/Fe] composition of a SSP ejecta increases with metallicity.  When strong galactic outflows (and inflows) are present, [Fe/H] will increase more slowly.  As a consequence, more metal-poor stars will be formed and the stellar models at high metallicity will not be used, thus decreasing the [Mn/Fe] abundances of the galactic gas. 